Import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
import json
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
import tensorflowjs as tfjs

nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Jacopo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Pre-Processor

In [2]:
def remove_stopwords(input_text):
    stopwords_list = stopwords.words('english')
    stopwords_list.append('rt')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 
    
def remove_mentions(input_text):
    return re.sub(r'@\w+', '', input_text)

def pre_process(input_text):
    output = input_text.apply(lambda x: x.lower())
    output = output.apply(remove_stopwords).apply(remove_mentions)
    output = output.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    return output

Load and Pre-Process data

In [3]:
#download from kaggle
#!curl https://www.kaggle.com/crowdflower/first-gop-debate-twitter-sentiment/downloads/Sentiment.csv/2 > Sentiment.csv
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
# Easy: binary classifier
data = data[data.sentiment != "Neutral"]

data['text'] = pre_process(data['text'])

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
data.head()

/Users/Jacopo/anaconda/envs/TFJS/lib/python2.7/site-packages/ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys


4472
16986


,text,sentiment
1,catch full gopdebate last night scotts best l...,Positive
3,carly fiorina trending hours debate men jus...,Positive
4,gopdebate w delivered highest ratings histor...,Positive
5,on first day rescind every illegal executive...,Positive
6,liked happy heard going moderator not anymore...,Negative


Tokenizer

In [4]:
max_length = 50 # max number of words in a sentence

tokenizer = Tokenizer() # num_words=2000, split=' ', char_level=True
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=max_length, padding='post')

Prepare Embedding

In [5]:
# download GloVe
#!curl http://nlp.stanford.edu/data/glove.6B.zip > glove.6B.zip

# Prepare Glove File
def readGloveFile(gloveFile):
    with open(gloveFile, 'r') as f:
        wordToGlove = {}  # map from a token (word) to a Glove embedding vector
        wordToIndex = {}  # map from a token to an index
        indexToWord = {}  # map from an index to a token 

        for line in f:
            record = line.strip().split()
            token = record[0] # take the token (word) from the text line
            wordToGlove[token] = np.array(record[1:], dtype=np.float64) # associate the Glove embedding vector to a that token (word)

        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1  # 0 is reserved for masking in Keras (see above)
            wordToIndex[tok] = kerasIdx # associate an index to a token (word)
            indexToWord[kerasIdx] = tok # associate a word to a token (word). Note: inverse of dictionary above

    return wordToIndex, indexToWord, wordToGlove


# Create Pretrained Keras Embedding Weights Matrix
def createPretrainedEmbeddingMatrix(wordToGlove, wordToIndex):
    vocabLen = len(wordToIndex) + 1  # adding 1 to account for masking
    embDim = next(iter(wordToGlove.values())).shape[0]  # works with any glove dimensions (e.g. 50)

    embeddingMatrix = np.zeros((vocabLen, embDim))  # initialize with zeros
    for word, index in wordToIndex.items():
        vector = wordToGlove.get(word) # create embedding: word index to Glove word embedding
        if vector is not None:        
            embeddingMatrix[index, :] = vector

    return vocabLen, embDim, embeddingMatrix


wordToIndex, indexToWord, wordToGlove = readGloveFile("glove.6B.100d.txt")
#vocabLen, embDim, embeddingMatrix = createPretrainedEmbeddingMatrix(wordToGlove, wordToIndex)
vocabLen, embDim, embeddingMatrix = createPretrainedEmbeddingMatrix(wordToGlove, tokenizer.word_index)
embedding = Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=max_length, trainable=False) #trainable=False

Create Model

In [6]:
lstm_out = 196
model = Sequential()
model.add(embedding)
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           1275800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               232848    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 1,509,042
Trainable params: 233,242
Non-trainable params: 1,275,800
_________________________________________________________________
None


Train Model

In [7]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

((7188, 50), (7188, 2))
((3541, 50), (3541, 2))
Epoch 1/7
 - 14s - loss: 0.5284 - acc: 0.7881
Epoch 2/7
 - 13s - loss: 0.5185 - acc: 0.7891
Epoch 3/7
 - 13s - loss: 0.5171 - acc: 0.7891
Epoch 4/7
 - 14s - loss: 0.5173 - acc: 0.7891
Epoch 5/7
 - 15s - loss: 0.5164 - acc: 0.7891
Epoch 6/7
 - 15s - loss: 0.5161 - acc: 0.7891
Epoch 7/7
 - 15s - loss: 0.5169 - acc: 0.7891


Evaluate Model

In [8]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.51
acc: 0.80


Test Model

In [9]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#twt = ['We all are great people']


#pre-processing
twt = pd.DataFrame({'text':twt})
twt['text'] = pre_process(twt['text'])
twt = twt['text'].tolist()
print(twt)

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=max_length, dtype='int32', value=0)
print(twt)

sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['meetings none us dumb us']
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0 504  77 908  77]]
[ 0.80033928  0.1996607 ]
negative


Export Model and Metadata

In [10]:
with open('words.json', 'w') as f:
    f.write(json.dumps(tokenizer.word_index))

tfjs.converters.save_keras_model(model, 'tfjsmodel')